# Read Dimension of Remuneration

役員報酬のデータを収集する


In [11]:
import os
from pathlib import Path
import edinet

# NTTドコモ/R1.06.19 15:00
DOC_ID = "S100G1DL"

# Data Folder
DATA_ROOT = Path.cwd().joinpath("data")

## Download XBRL

In [12]:
from edinet.xbrl_file import XBRLDir


# Download and load document
xbrl_path = edinet.api.document.get_xbrl(
    DOC_ID, save_dir=DATA_ROOT.joinpath("raw"),
    expand_level="dir")

xbrl_dir = XBRLDir(xbrl_path)

c:\Users\ico\Documents\works\xbrl_read_tutorial\venv\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [13]:
xbrl = xbrl_dir.xbrl._root

## Find tags with same dimension and member



In [14]:
from lxml import etree
import pandas as pd


remuneration_tag ="TotalAmountOfRemunerationEtcRemunerationEtcByCategoryOfDirectorsAndOtherOfficers"

data = []
for t in xbrl.find_all(remuneration_tag):
    context_id = t["contextRef"]
    context = xbrl.find("xbrli:context", {"id": context_id})
    period = context.find("xbrli:endDate").text
    period_begin = context.find("xbrli:startDate").text
    member = context.find("xbrldi:explicitMember", {"dimension": "jpcrp_cor:CategoriesOfDirectorsAndOtherOfficersAxis"}).text

    for t in xbrl.select(f"[contextRef='{context_id}']"):
        d = {
            "member": member,
            "element": t.name,
            "period": period,
            "period_begin": period_begin,
            "amount": t.text
        }
        data.append(d)

remunerations = pd.DataFrame(data)

In [15]:
remunerations.head(5)

,member,element,period,period_begin,amount
0,jpcrp_cor:DirectorsExcludingOutsideDirectorsMe...,TotalAmountOfRemunerationEtcRemunerationEtcByC...,2019-03-31,2018-04-01,481000000
1,jpcrp_cor:DirectorsExcludingOutsideDirectorsMe...,FixedRemunerationRemunerationEtcByCategoryOfDi...,2019-03-31,2018-04-01,336000000
2,jpcrp_cor:DirectorsExcludingOutsideDirectorsMe...,PerformanceBasedRemunerationRemunerationEtcByC...,2019-03-31,2018-04-01,145000000
3,jpcrp_cor:DirectorsExcludingOutsideDirectorsMe...,RetirementBenefitsRemunerationEtcByCategoryOfD...,2019-03-31,2018-04-01,
4,jpcrp_cor:DirectorsExcludingOutsideDirectorsMe...,NumberOfDirectorsAndOtherOfficersRemunerationE...,2019-03-31,2018-04-01,16
